In [1]:
%pip install requests

In [ ]:
%pip install chromadb

In [ ]:
%pip install sentence_transformers

In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [64]:
%pip install tqdm

## Importando dados

In [66]:
from tqdm import tqdm

In [1]:
from huggingface_hub import login
from huggingface_hub import HfApi
from datasets import load_dataset
from datasets import Dataset
from unsloth import FastLanguageModel
import torch
import pandas as pd
import os
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from transformers import pipeline

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [67]:
tqdm.pandas()

In [3]:
login(token="seu token do hugging face vai aqui")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### dataset de teste

In [4]:
import pandas as pd

splits = {'train': 'data/train.parquet', 'test': 'data/test.parquet'}
df_test = pd.read_parquet("hf://datasets/matiusX/legislacao-ufam/" + splits["test"])

### banco de dados

In [5]:
!git clone https://github.com/matiusX/UFAM-LegisAssistant.git

fatal: destination path 'UFAM-LegisAssistant' already exists and is not an empty directory.


In [6]:
COLLECTION_NAME = "legislacao"
CHROMA_CLIENT = chromadb.PersistentClient(path="/content/UFAM-LegisAssistant/legislacao")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")
COLLECTION = CHROMA_CLIENT.get_or_create_collection(name=COLLECTION_NAME, embedding_function=sentence_transformer_ef)

In [7]:
#retorna o conexto para uma pergunta
def context(question, top_k = 1):
    results = COLLECTION.query(
        query_texts=[question],
        n_results= top_k,
        include=['documents']
    )
    combined_string = ' '.join(results["documents"][0])
    return combined_string

In [8]:
#exemplo de uso
context('Como os funcionários públicos são envolvidos nas decisões administrativas?')

'REGIMENTO GERAL\n\nArt. 1º. O presente Regimento Geral iscinina os aspectos de organização e funcionamento\ncomuns aos vários órgãos e serviços da Universidade Federal do Amazonas, cuj Estao completa.\n\nParágato único - As normas deste Regimenta serão complementadas pelos\nregimentos da Reto, das Unidades Acadêmicas e de ouros órgãos, na que devam compreender\nde especíico\n\ntíruLo!\n\nDa Administração Universitária\n\nA 2º A administração da Universidade dar-se à em nivel suprir em nível\ndas Unidades Acadêmica, através dos respeivos Órgãos delberaivos e executo\n\nArt. 3º. Administração Superior será exercia:\n\n1 pelo Conselho Univesstáro - CONSUNI, com funções desberaivas e\nnormativas;\n\nH pelo Conseho de Adminsração - CONSAD, com funções consultvas,\ndeleratvas e normativas;\n\nML. polo Conselho de Ensino, Pesquisa e Extensão — CONSEPE, com funções.\nconsulvas, delberativas e normaivas\n\nMM. pela Reitoria, com função executiva.\n\nArt. 4º - A administração das Unidades Acadê

In [9]:
#O contexto já foi pré-processado no dataframe df_test
df_test

,perguntas,respostas,contexto
0,Como os funcionários públicos são envolvidos n...,Os funcionários públicos são envolvidos nas de...,REGIMENTO GERAL\n\nArt. 1º. O presente Regimen...
1,Como a UFAM incentiva a participação dos disce...,A UFAM incentiva a participação dos discentes ...,Regulamenta o uso do nome social no âmbito da ...
2,Quais são as exigências para o credenciamento ...,As exigências para o credenciamento de institu...,Presidência da República\nCasa Civil\nSubchefi...
3,Como a UFAM assegura a transparência no proces...,A UFAM assegura a transparência no processo de...,(6) UNIVERSIDADE FEDERAL DO AMAZONAS\nTU; Cons...
4,O que define o Regimento Geral da UFAM?,O Regimento Geral da UFAM define os aspectos d...,REGIMENTO GERAL\n\nArt. 1º. O presente Regimen...
...,...,...,...
813,Qual é o impacto das políticas da Reitoria na ...,As políticas da Reitoria impactam a UFAM ao de...,Presidência da República\nCasa Civi\nSubchefia...
814,Quem aprovou a moção de apoio ao PET?,A moção de apoio ao PET foi aprovada pelo Cons...,"DECRETO N.' 2.561, DE 27 DE AbrIL dE 1998\nAlt..."
815,De que maneira a LDB promove a pesquisa e a in...,A LDB promove a pesquisa e a inovação nas univ...,"Prezado Senhor.\nEm atenção à sua solicitação,..."
816,Qual é o prazo para a comissão concluir a anál...,A comissão tem o prazo de 15 dias para conclui...,"O PRESIDENTE DA REPÚBLICA, no uso da atribuiçã..."


## Utilizando o modelo

In [10]:
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "matiusX/lamma-legis-ufam",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.4.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [12]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [56]:
def gera_resposta_dataset(row):
  inputs = tokenizer(
  [
      prompt.format(
          row["contexto"][:10000], # contexto
          row["perguntas"], # pergunta
          "", # resposta - deixar em branco
      )
  ], return_tensors = "pt").to("cuda")

  # Gerar a saída do modelo
  outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

  # Decodificar a saída
  output_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

  # Capturar o texto gerado numa variável
  generated_text = output_texts[0]

  # Encontrar a posição de "response :" e pegar o texto a partir daí
  start_index = generated_text.find("Response:")
  if start_index != -1:
      generated_text = generated_text[start_index + len("Response:"):].strip()

  return generated_text



In [68]:
#gera campo de respostas_geradas
df_test['respostas_geradas'] = df_test.progress_apply(gera_resposta_dataset, axis=1)

100%|██████████| 818/818 [1:56:04<00:00,  8.51s/it]


In [69]:
df_test.to_parquet('final.parquet', index=False)